In [1]:
import json
import pickle
import numpy as np
import pandas as pd
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn,Tensor
from typing import Union, Tuple, List, Iterable, Dict

In [2]:
msf_data = pd.read_csv('./msf_baby_bonus.csv', header = None)

In [3]:
test2_data = pd.read_csv('./aprilfinal.csv')
test2_data['original']  =test2_data['original'].apply(lambda x : x.replace('\n',''))
test2_data['reframed']  =test2_data['reframed'].apply(lambda x : x.replace('\n',''))

In [4]:
with open('output.json') as f:
    generated_ques = json.load(f)

In [5]:
with open('test_answers.txt', 'r') as f:
    test_answers = f.readlines()
    test_answers = list(map(lambda x : x.replace('\n',''),test_answers))
    test_answers = list(filter(lambda x : len(x) > 0 , test_answers))


In [6]:
with open('test_questions.txt', 'r') as f:
    test_questions = f.readlines()
    test_questions  = list(map(lambda x : x.replace('\n',''),test_questions))
    test_questions  = list(filter(lambda x : len(x) > 0 , test_questions))


In [7]:
with open('answers.txt', 'r', encoding="utf8") as f:
    answers = f.readlines()
    answers  = list(map(lambda x : x.replace('\n',''),answers))
    answers  = list(filter(lambda x : len(x) > 0 , answers))

    

In [8]:
csv_answers = msf_data[2].tolist()
csv_answers = list(map(lambda x : x.replace('\n', '').strip(),csv_answers))

In [9]:
csv_questions = msf_data[1].tolist()
csv_questions = list(map(lambda x : x.replace('\n', '').strip(),csv_questions))

In [10]:
def make_dataset():
    questions = []
    labels = []
    for q,a in zip(csv_questions , csv_answers):
        if(a not in answers):
            continue
        
        questions.append(q)
        labels.append(answers.index(a))
    return questions , labels
        
        
    

In [11]:
questions , labels = make_dataset()

In [12]:
import sys
sys.path.append('./scripts')
sys.path.append('./')

In [13]:
from modelInterface import modelInterface

In [14]:
modelInter = modelInterface(faq_name="baby_bonus")

AssertionError: Did not find and preexisting of baby_bonus so you must provide faq_data

In [15]:
test_questions_labels_dict  = {}
data_questions_label_dict = {}
for q,l in zip(questions, labels):
    data_questions_label_dict[q] = l
    gens = generated_ques[q]
  #  gens += aug(q,10)
    for gen in gens:
        data_questions_label_dict[gen] = l
for q,l in zip(test_questions, test_answers):
    test_questions_labels_dict[q] = l
    
answer_to_label = {ans: i for i,ans in enumerate(answers)}

In [16]:
new_data = {'question_to_label': data_questions_label_dict , 'answer_to_label': answer_to_label}

In [17]:
len(data_questions_label_dict)

3134

In [18]:
modelInter.train("/checkpoints/model5")

./scripts\utils.py:111: UserWarning: k is greater than than number of generated ques
  warnings.warn('k is greater than than number of generated ques')


fitting now


tensor(0.2781, grad_fn=<MeanBackward0>)
tensor(0.0056, grad_fn=<MeanBackward0>)
tensor(0.1709, grad_fn=<MeanBackward0>)
tensor(0.0555, grad_fn=<MeanBackward0>)
tensor(0.0094, grad_fn=<MeanBackward0>)
tensor(0.2182, grad_fn=<MeanBackward0>)
tensor(0.1052, grad_fn=<MeanBackward0>)
tensor(0.0774, grad_fn=<MeanBackward0>)
tensor(0.0691, grad_fn=<MeanBackward0>)
tensor(1.4614, grad_fn=<MeanBackward0>)
tensor(0.0737, grad_fn=<MeanBackward0>)
tensor(0.2129, grad_fn=<MeanBackward0>)
tensor(0.4021, grad_fn=<MeanBackward0>)
tensor(0.0830, grad_fn=<MeanBackward0>)
tensor(0.2515, grad_fn=<MeanBackward0>)
tensor(0.2384, grad_fn=<MeanBackward0>)
tensor(0.2252, grad_fn=<MeanBackward0>)
tensor(0.9226, grad_fn=<MeanBackward0>)
tensor(0.1251, grad_fn=<MeanBackward0>)
tensor(0.3954, grad_fn=<MeanBackward0>)
tensor(0.0115, grad_fn=<MeanBackward0>)
tensor(0.1159, grad_fn=<MeanBackward0>)
tensor(1.1295, grad_fn=<MeanBackward0>)
tensor(0.6568, grad_fn=<MeanBackward0>)
tensor(0.2347, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 

In [18]:
modelInter.fit_FAQ()

261 label present in answer but not in question


./scripts\modelInterface.py:327: UserWarning: some labels in answers are not present in questions , you might not have labels in Sync
  warnings.warn('some labels in answers are not present in questions , you might not have labels in Sync')


In [1]:
modelInter.fit_FAQ(data_questions_label_dict, answer_to_label)

NameError: name 'modelInter' is not defined

In [21]:
modelInter.answer_question('what bank should I use ?', K = 1, cutoff = .5)

0.5702707767486572
Answering When will CDA bank record be updated to show my savings deposited recently?


'The Child Development Account (CDA) banks will update MSF with your deposit information on a weekly basis. You should be able to view your CDA savings by the following Tuesday on the Baby Bonus Online portal. Alternatively, you can use your bank’s internet banking services to check your savings in the CDA.'

In [17]:
modelInter.train({'questions':questions , 'generated_ques': generated_ques, 'labels':labels , 'bs':64 ,'n':8},model_save_path =os.path.join('checkpoints', 'myModel4') )

fitting now


tensor([198, 198, 198, 198, 198, 198, 198, 198, 184, 184, 184, 184, 184, 184,
        184, 184,  48,  48,  48,  48,  48,  48,  48,  48, 206, 206, 206, 206,
        206, 206, 206, 206,  97,  97,  97,  97,  97,  97,  97,  97, 107, 107,
        107, 107, 107, 107, 107, 107,  33,  33,  33,  33,  33,  33,  33,  33,
        124, 124, 124, 124, 124, 124, 124, 124], dtype=torch.int32)
tensor(1.1054, grad_fn=<MeanBackward0>)
tensor([175, 175, 175, 175, 175, 175, 175, 175,  39,  39,  39,  39,  39,  39,
         39,  39, 255, 255, 255, 255, 255, 255, 255, 255,  78,  78,  78,  78,
         78,  78,  78,  78,  88,  88,  88,  88,  88,  88,  88,  88, 258, 258,
        258, 258, 258, 258, 258, 258,  64,  64,  64,  64,  64,  64,  64,  64,
         21,  21,  21,  21,  21,  21,  21,  21], dtype=torch.int32)
tensor(0.4469, grad_fn=<MeanBackward0>)
tensor([156, 156, 156, 156, 156, 156, 156, 156, 224, 224, 224, 224, 224, 224,
        224, 224, 154, 154, 154, 154, 154, 154, 154, 154, 178, 178, 178, 178,
    

tensor([220, 220, 220, 220, 220, 220, 220, 220, 239, 239, 239, 239, 239, 239,
        239, 239, 176, 176, 176, 176, 176, 176, 176, 176,  76,  76,  76,  76,
         76,  76,  76,  76, 251, 251, 251, 251, 251, 251, 251, 251, 178, 178,
        178, 178, 178, 178, 178, 178,  63,  63,  63,  63,  63,  63,  63,  63,
        279, 279, 279, 279, 279, 279, 279, 279], dtype=torch.int32)
tensor(0.5358, grad_fn=<MeanBackward0>)
tensor([ 87,  87,  87,  87,  87,  87,  87,  87, 282, 282, 282, 282, 282, 282,
        282, 282,   1,   1,   1,   1,   1,   1,   1,   1, 275, 275, 275, 275,
        275, 275, 275, 275,   5,   5,   5,   5,   5,   5,   5,   5, 269, 269,
        269, 269, 269, 269, 269, 269,  84,  84,  84,  84,  84,  84,  84,  84,
        252, 252, 252, 252, 252, 252, 252, 252], dtype=torch.int32)
tensor(0.6907, grad_fn=<MeanBackward0>)
tensor([276, 276, 276, 276, 276, 276, 276, 276, 219, 219, 219, 219, 219, 219,
        219, 219, 171, 171, 171, 171, 171, 171, 171, 171, 237, 237, 237, 237,
    

tensor([213, 213, 213, 213, 213, 213, 213, 213, 237, 237, 237, 237, 237, 237,
        237, 237, 193, 193, 193, 193, 193, 193, 193, 193, 105, 105, 105, 105,
        105, 105, 105, 105, 207, 207, 207, 207, 207, 207, 207, 207, 281, 281,
        281, 281, 281, 281, 281, 281, 254, 254, 254, 254, 254, 254, 254, 254,
        158, 158, 158, 158, 158, 158, 158, 158], dtype=torch.int32)
tensor(0.7917, grad_fn=<MeanBackward0>)
tensor([128, 128, 128, 128, 128, 128, 128, 128,  66,  66,  66,  66,  66,  66,
         66,  66, 237, 237, 237, 237, 237, 237, 237, 237, 279, 279, 279, 279,
        279, 279, 279, 279, 275, 275, 275, 275, 275, 275, 275, 275,  41,  41,
         41,  41,  41,  41,  41,  41, 191, 191, 191, 191, 191, 191, 191, 191,
        125, 125, 125, 125, 125, 125, 125, 125], dtype=torch.int32)
tensor(0.4497, grad_fn=<MeanBackward0>)
tensor([281, 281, 281, 281, 281, 281, 281, 281, 160, 160, 160, 160, 160, 160,
        160, 160,  28,  28,  28,  28,  28,  28,  28,  28,   6,   6,   6,   6,
    

tensor([248, 248, 248, 248, 248, 248, 248, 248, 170, 170, 170, 170, 170, 170,
        170, 170,  65,  65,  65,  65,  65,  65,  65,  65,  91,  91,  91,  91,
         91,  91,  91,  91,   7,   7,   7,   7,   7,   7,   7,   7, 184, 184,
        184, 184, 184, 184, 184, 184, 288, 288, 288, 288, 288, 288, 288, 288,
        260, 260, 260, 260, 260, 260, 260, 260], dtype=torch.int32)
tensor(0.1244, grad_fn=<MeanBackward0>)
tensor([255, 255, 255, 255, 255, 255, 255, 255,   9,   9,   9,   9,   9,   9,
          9,   9, 258, 258, 258, 258, 258, 258, 258, 258, 251, 251, 251, 251,
        251, 251, 251, 251, 167, 167, 167, 167, 167, 167, 167, 167, 213, 213,
        213, 213, 213, 213, 213, 213, 208, 208, 208, 208, 208, 208, 208, 208,
         77,  77,  77,  77,  77,  77,  77,  77], dtype=torch.int32)
tensor(0.7357, grad_fn=<MeanBackward0>)
tensor([124, 124, 124, 124, 124, 124, 124, 124,  88,  88,  88,  88,  88,  88,
         88,  88, 277, 277, 277, 277, 277, 277, 277, 277, 121, 121, 121, 121,
    

tensor([104, 104, 104, 104, 104, 104, 104, 104,  33,  33,  33,  33,  33,  33,
         33,  33,  53,  53,  53,  53,  53,  53,  53,  53, 184, 184, 184, 184,
        184, 184, 184, 184, 142, 142, 142, 142, 142, 142, 142, 142, 267, 267,
        267, 267, 267, 267, 267, 267,  11,  11,  11,  11,  11,  11,  11,  11,
        228, 228, 228, 228, 228, 228, 228, 228], dtype=torch.int32)
tensor(0.6691, grad_fn=<MeanBackward0>)
tensor([129, 129, 129, 129, 129, 129, 129, 129,  99,  99,  99,  99,  99,  99,
         99,  99, 231, 231, 231, 231, 231, 231, 231, 231,  88,  88,  88,  88,
         88,  88,  88,  88, 162, 162, 162, 162, 162, 162, 162, 162,  79,  79,
         79,  79,  79,  79,  79,  79,  45,  45,  45,  45,  45,  45,  45,  45,
        270, 270, 270, 270, 270, 270, 270, 270], dtype=torch.int32)
tensor(0.2202, grad_fn=<MeanBackward0>)
tensor([203, 203, 203, 203, 203, 203, 203, 203,  44,  44,  44,  44,  44,  44,
         44,  44,  64,  64,  64,  64,  64,  64,  64,  64,  53,  53,  53,  53,
    

KeyboardInterrupt: 

In [43]:
data = {'data_questions_labels_dict': data_questions_label_dict , 'test_questions_labels_dict' : test_questions_labels_dict}

In [62]:
test2_questions_labels_dict = {test_que: data_questions_label_dict[que] for test_que, que in zip(test2_data['reframed'], test2_data['original']) if que in questions }


In [63]:
data2 = {'data_questions_labels_dict': data_questions_label_dict , 'test_questions_labels_dict' : test2_questions_labels_dict}

In [68]:
modelInter.evaluate(data2)

0.9621993127147767

In [49]:
#Augmentation start

In [181]:
class AUG():
    def __init__(self):
        import nlpaug.augmenter.char as nac
        import nlpaug.augmenter.word as naw
        aug0 = naw.RandomWordAug()
        aug1 = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
        aug2 = naw.SynonymAug(aug_src='wordnet')
        aug3 = naw.SplitAug()
        aug4 = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

        self.augs = [aug0, aug1,aug2,aug3,aug4]
        
    def __call__(self,sent,n= 1):
        return self.augment(sent,n)
        
    
    def augment(self,sent ,n = 1):
        import random
        ans = []
        sent = re.sub(r'[^a-zA-Z0-9_ ]', '', sent)
        for _ in range(n):
            aug = random.choice(self.augs)
            ans.append(aug.augment(sent))
                    
                
        return ans
       
        
        
        
        
            
        
        
    